<a href="https://colab.research.google.com/github/harry-erskine/PRBX-Travel-Mode-Identification/blob/main/Source%20Code/Pre-Processing/Pre_Processing_Steps_1_to_3_Extract%2C_Label%2C_and_Clean_the_Geolife_GPS_Dataset_into_a_Pandas_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Pre-Processing - Steps 1 to 3: Extract, Label, and Clean the Geolife GPS Dataset into a Pandas DataFrame**

This notebook will contain only the crucial blocks from steps 1 to 3 (so will not include some of the extra code that I have written to test or analyse my solutions).

The purpose of this notebook is to convert the Geolife GPS Dataset to the point before we group the trajectories.

where necessary, please change the variables for 'yourMountPath' and 'yourDrivePath' as well as uncommenting the installs in the 'Notebook Setup' section.

These steps will take about 25 minutes.

### **Notebook Setup**

In [ ]:
###
### If you were to run this notebook for yourself,
### after you have copied the whole 'PRBX' root folder correctly,
### the strings 'yourDrivePath' and 'yourMountPath'
### would be the only variables thatyou would need to change
### within this entire notebook for the rest of the code to work.
###

yourMountPath = ''  # add where you are mounting your drive here
yourDrivePath = ''  # add your drive path here

# these are the drive paths which I have been using
mainMountPath = '/content/gdrive'
mainDrivePath = '/content/gdrive/MyDrive/Colab Notebooks/PRBX'

In [ ]:
### INSTALLS ###

# !pip install --upgrade gspread
# !pip install pandas
# !pip install geopy

In [ ]:
### IMPORTS ###

import os
from google.colab import drive
import pandas as pd
from datetime import datetime
import time
import numpy as np

In [ ]:
### MOUNT GOOGLE DRIVE ###

# This will make yourDrivePath the path we use instead (if that has been filled in)
if yourMountPath != '':
  drive.mount(yourMountPath, force_remount=True)
else:
  drive.mount(mainMountPath, force_remount=True)

if yourDrivePath != '':
  mainDrivePath = yourDrivePath

Mounted at /content/gdrive


In [ ]:
### FILE PATHS ###

trajectoriesPath = mainDrivePath + '/Data Files/Geolife Trajectories 1.3/Data'
generatedFilesPath = mainDrivePath + '/Data Files/Generated Files'
# labelsPath = generatedFilesPath + '/labels.csv'
# filteredTrajectoriesPath = generatedFilesPath + '/filteredTrajectories.csv'
# labelledTrajectoriesPath = generatedFilesPath + '/labelledTrajectories.csv'
cleanedLabelledTrajectoriesPath = generatedFilesPath + '/cleanedLabelledTrajectories.csv'

### **Step 1: Pre-Processing - Convert Data Files into a Pandas DataFrames**

This step will take about 18 mins

In [ ]:
# List all files and directories in the mounted directory
files_and_directories = os.listdir(mainDrivePath)

# Print the list
print("\nFiles and directories in the mounted Google Drive:")
for item in files_and_directories:
    print(item)

files_and_directories = os.listdir(trajectoriesPath)
countOfTrajectories = 0

#print("\nNames of Trajectories Folders:")
for item in files_and_directories:
    #print(item)
    countOfTrajectories += 1

print("\nNumber of Trajectories:", countOfTrajectories)


Files and directories in the mounted Google Drive:
Data Files
Source Code

Number of Trajectories: 182


In [ ]:
def getTimestamp(date, time, seperator):
  dateValues = date.split(seperator) # seperator would be either '/' or '-'
  timeValues = time.split(':')
  values = []
  for v in dateValues:
    values.append(int(v))
  for v in timeValues:
    values.append(int(v))
  datetimeValue = datetime(values[0],values[1],values[2],values[3],values[4],values[5])
  return int(datetimeValue.timestamp())

In [ ]:
# this will return a list of all of the users (formatted correctly)
# so that their path can be called upon correctly

numUsers = 182

def generateUsers():
  users = []
  for i in range(10):
    users.append('00' + str(i))
  for i in range(10, 100):
    users.append('0' + str(i))
  for i in range(100,182):
    users.append(str(i))
  return users

users = generateUsers()

In [ ]:
labelledUsers = []
labelsData = []

for i in range(numUsers):

  originalLabelsPath = trajectoriesPath + '/' + users[i] + '/labels.txt'
  # print("At user: " + users[i])

  if os.path.isfile(originalLabelsPath):

    labelledUsers.append(users[i])

    with open(originalLabelsPath) as contents:
      records = contents.readlines()

    numLines = len(records)
    for r in records[1:]:
      rVals = r.split()
      startTime = getTimestamp(rVals[0], rVals[1], '/')
      endTime = getTimestamp(rVals[2], rVals[3], '/')
      modeType = rVals[4]
      labelsData.append([users[i], startTime, endTime, modeType])

In [ ]:
# will be using the unix timestamp here
labelsDf = pd.DataFrame(labelsData, columns=['UserId', 'Start', 'End', 'Mode'])
labelsDf.head()

,UserId,Start,End,Mode
0,010,1182857549,1182858029,bus
1,010,1206715974,1206719999,train
2,010,1206720000,1206741720,train
3,010,1206754070,1206806399,train
4,010,1206806400,1206892799,train


In [ ]:
# save labelsDf to the labels file path (as a .csv)
# labelsDf.to_csv(labelsPath, index=False)

In [ ]:
# unix start date
date_1970 = datetime(1970, 1, 1)

# start date for field 5 of our data
date_1899 = datetime(1899, 12, 30)

secs_between_1989_and_1970 = (date_1970 - date_1899).total_seconds()

print("Number of seconds between the two dates:", secs_between_1989_and_1970)

Number of seconds between the two dates: 2209161600.0


In [ ]:
def getTimestampVersion2(daysAfter1899):
  secsAfter1899 = float(daysAfter1899) * 60 * 60 * 24
  secsAfter1970 = secsAfter1899 - secs_between_1989_and_1970
  return secsAfter1970

In [ ]:
# Start timing this process (Will take about 18 minutes)
total_start_time = time.time()

trajData = []

for i in range(len(labelledUsers)):

  trajRootPath = trajectoriesPath + '/' + labelledUsers[i] + '/Trajectory'

  # print(f"At user: {labelledUsers[i]}    - time elapsed: {time.time() - total_start_time:.2f} seconds")

  for file in os.listdir(trajRootPath):

    filePath = trajRootPath + '/' + file
    # print("At file: " + file + " which has path: " + filePath)

    with open(filePath) as contents:
      records = contents.readlines()

    for r in records[6:]:
      rVals = r.split(',')
      latitude = rVals[0]
      longitude = rVals[1]
      # rVal[2] is not used
      altitude = rVals[3]
      timestampVersion2 = getTimestampVersion2(rVals[4])
      timestamp = getTimestamp(rVals[5], rVals[6], '-')
      trajData.append([labelledUsers[i], latitude, longitude, altitude, timestamp, timestampVersion2])
      #print(trajData[-1])

time_in_mins = (time.time() - total_start_time) / 60
print(f"\nTotal processing time: {time_in_mins:.2f} minutes")


Total processing time: 4.17 minutes


In [ ]:
trajDf = pd.DataFrame(trajData, columns=['UserId', 'Latitude', 'Longitude', 'Altitude', 'Timestamp', 'Timestamp V2'])
trajDf

,UserId,Latitude,Longitude,Altitude,Timestamp,Timestamp V2
0,010,39.12299,117.244615,85,1188506368,1.188506e+09
1,010,39.12299,117.244615,85,1188506368,1.188506e+09
2,010,39.123213,117.243748,85,1188506374,1.188506e+09
3,010,39.123238,117.243627,85,1188506375,1.188506e+09
4,010,39.123277,117.243487,85,1188506376,1.188506e+09
...,...,...,...,...,...,...
12548230,179,40.087042,116.304183,191,1226906064,1.226906e+09
12548231,179,40.087036,116.304177,192,1226906066,1.226906e+09
12548232,179,40.087028,116.304172,193,1226906068,1.226906e+09
12548233,179,40.08702,116.304167,194,1226906070,1.226906e+09


In [ ]:
# Drop the 'Timestamp V2' column from trajDf
trajDf.drop(columns=['Timestamp V2'], inplace=True)
trajDf

,UserId,Latitude,Longitude,Altitude,Timestamp
0,010,39.12299,117.244615,85,1188506368
1,010,39.12299,117.244615,85,1188506368
2,010,39.123213,117.243748,85,1188506374
3,010,39.123238,117.243627,85,1188506375
4,010,39.123277,117.243487,85,1188506376
...,...,...,...,...,...
12548230,179,40.087042,116.304183,191,1226906064
12548231,179,40.087036,116.304177,192,1226906066
12548232,179,40.087028,116.304172,193,1226906068
12548233,179,40.08702,116.304167,194,1226906070


In [ ]:
# save trajDf to the filtered trajectories file path (as a .csv)
# trajDf.to_csv(filteredTrajectoriesPath, index=False)

### **Step 2: Pre-Processing - Label all Trajectories into a Single DataFrame**

This step will take about 5 mins

In [ ]:
# labelsDf = pd.read_csv(labelsPath)
labelsDf = labelsDf.sort_values(by=['UserId', 'Start'])
labelsDf

,UserId,Start,End,Mode
0,010,1182857549,1182858029,bus
1,010,1206715974,1206719999,train
2,010,1206720000,1206741720,train
3,010,1206754070,1206806399,train
4,010,1206806400,1206892799,train
...,...,...,...,...
14713,179,1226905198,1226905576,bus
14714,179,1226905576,1226906072,walk
14715,179,1227923885,1227924099,bus
14716,179,1227924099,1227924477,walk


In [ ]:
# trajDf = pd.read_csv(filteredTrajectoriesPath)
trajDf = trajDf.sort_values(by=['UserId', 'Timestamp'])
trajDf

,UserId,Latitude,Longitude,Altitude,Timestamp
73397,010,39.921712,116.472343,13,1186198232
73398,010,39.921705,116.472343,13,1186198233
73399,010,39.921695,116.472345,13,1186198234
73400,010,39.921683,116.472342,13,1186198235
73401,010,39.921672,116.472342,13,1186198236
...,...,...,...,...,...
12545461,179,40.007802,116.319362,84,1227946552
12545462,179,40.00778,116.31936,88,1227946554
12545463,179,40.007756,116.319362,92,1227946556
12545464,179,40.00774,116.319361,97,1227946558


In [ ]:
labelledUsers = labelsDf['UserId'].unique()
print(labelledUsers)

['010' '020' '021' '052' '053' '056' '058' '059' '060' '062' '064' '065'
 '067' '068' '069' '073' '075' '076' '078' '080' '081' '082' '084' '085'
 '086' '087' '088' '089' '091' '092' '096' '097' '098' '100' '101' '102'
 '104' '105' '106' '107' '108' '110' '111' '112' '114' '115' '116' '117'
 '118' '124' '125' '126' '128' '129' '136' '138' '139' '141' '144' '147'
 '153' '154' '161' '163' '167' '170' '174' '175' '179']


In [ ]:
# print(len(labelledUsers)) # it's 69

In [ ]:
# This process will take about 5 minutes
total_start_time = time.time()

# Initialize LTD (Labelled Trajectories Data)
LTD = []

for u in labelledUsers:

    # print(f"At user: {u}    - time elapsed: {time.time() - total_start_time:.2f} seconds")

    # Filter trajectories and labels for the current user
    usersTrajDf = trajDf[trajDf['UserId'] == u]
    usersLabelsDf = labelsDf[labelsDf['UserId'] == u]

    # Create dictionaries to store time intervals and trajectories for the current user
    usersIntervals = list(zip(usersLabelsDf['Start'], usersLabelsDf['End'], usersLabelsDf['Mode']))
    usersTrajectories = list(zip(usersTrajDf['Timestamp'], usersTrajDf['Latitude'], usersTrajDf['Longitude'], usersTrajDf['Altitude']))

    currentLabelsIndex = 0

    # Iterate over trajectories for the current user
    for timestamp, latitude, longitude, altitude in usersTrajectories:

        # Iterate over time intervals for the current user
        for labelsIndex in range(currentLabelsIndex, len(usersIntervals)):

            start, end, mode = usersIntervals[labelsIndex]

            if start <= timestamp < end:

                LTD.append((u, mode, int(timestamp), float(latitude), float(longitude), float(altitude)))
                currentLabelsIndex = labelsIndex
                break  # Break out of the loop once the label is found

time_in_mins = (time.time() - total_start_time) / 60
print(f"Total processing time: {time_in_mins:.2f} minutes")

Total processing time: 6.76 minutes


In [ ]:
# create LTDF (labelled Trajectories DataFrame)
LTDF = pd.DataFrame(LTD, columns=['UserId', 'Mode', 'Timestamp', 'Latitude', 'Longitude', 'Altitude'])
LTDF

,UserId,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,train,1206716080,39.894178,116.318200,-777.0
1,010,train,1206716114,39.894505,116.321132,-777.0
2,010,train,1206716173,39.894953,116.326452,-777.0
3,010,train,1206716232,39.894600,116.332542,-777.0
4,010,train,1206716291,39.889622,116.337040,-777.0
...,...,...,...,...,...,...
5433664,179,subway,1227925767,40.029529,116.411977,291.0
5433665,179,subway,1227925769,40.029320,116.411975,289.0
5433666,179,subway,1227925771,40.029111,116.411963,275.0
5433667,179,subway,1227925773,40.028904,116.411962,274.0


In [ ]:
# save LTDF to the labelled trajectories file path (as a .csv)
# LTDF.to_csv(labelledTrajectoriesPath, index=False)

### **Step 3: Pre-Processing - Clean the Labelled Trajectories DataFrame**

This step will take just a minute

In [ ]:
# LTDF = pd.read_csv(labelledTrajectoriesPath)
LTDF

,UserId,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,train,1206716080,39.894178,116.318200,-777.0
1,010,train,1206716114,39.894505,116.321132,-777.0
2,010,train,1206716173,39.894953,116.326452,-777.0
3,010,train,1206716232,39.894600,116.332542,-777.0
4,010,train,1206716291,39.889622,116.337040,-777.0
...,...,...,...,...,...,...
5433664,179,subway,1227925767,40.029529,116.411977,291.0
5433665,179,subway,1227925769,40.029320,116.411975,289.0
5433666,179,subway,1227925771,40.029111,116.411963,275.0
5433667,179,subway,1227925773,40.028904,116.411962,274.0


In [ ]:
# Same function as used in 'Step 9'
def count_unique_modes(dataframe):
    # Find all unique values in the 'Mode' column
    unique_modes = dataframe['Mode'].unique()
    print("Unique Modes:", unique_modes, "\n")

    # Get the count of each mode
    mode_counts = dataframe['Mode'].value_counts()

    # Print unique values along with their counts
    for mode in unique_modes:
        count = mode_counts.get(mode, 0)
        print("Mode:", mode, "        - Count:", count)

    print("\nTotal count:", np.sum(mode_counts))

    return mode_counts

In [ ]:
unique_modes = count_unique_modes(LTDF)

Unique Modes: ['train' 'taxi' 'walk' 'bus' 'subway' 'airplane' 'car' 'bike' 'boat' 'run'
 'motorcycle'] 

Mode: train         - Count: 556343
Mode: taxi         - Count: 241056
Mode: walk         - Count: 1579984
Mode: bus         - Count: 1269700
Mode: subway         - Count: 309226
Mode: airplane         - Count: 9183
Mode: car         - Count: 512276
Mode: bike         - Count: 950033
Mode: boat         - Count: 3559
Mode: run         - Count: 1971
Mode: motorcycle         - Count: 338

Total count: 5433669


In [ ]:
# there is not enough data for the 'motorcycle' mode
# therefore I will drop all data with mode 'motorcycle'
LTDF = LTDF[LTDF['Mode'] != 'motorcycle']

In [ ]:
# First I will remove all data points where the altitude is -777 (as this means that this data wasn't recorded)
LTDF = LTDF[LTDF['Altitude'] != -777]
LTDF.reset_index(drop=True, inplace=True)
LTDF

,UserId,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,walk,1207116566,39.477125,75.989985,4096.0
1,010,walk,1207116568,39.477083,75.989990,4094.0
2,010,walk,1207116569,39.477053,75.989980,4094.0
3,010,walk,1207116570,39.477033,75.989972,4094.0
4,010,walk,1207116608,39.476977,75.989965,4101.0
...,...,...,...,...,...,...
5377865,179,subway,1227925767,40.029529,116.411977,291.0
5377866,179,subway,1227925769,40.029320,116.411975,289.0
5377867,179,subway,1227925771,40.029111,116.411963,275.0
5377868,179,subway,1227925773,40.028904,116.411962,274.0


In [ ]:
print(LTDF.dtypes)

UserId        object
Mode          object
Timestamp      int64
Latitude     float64
Longitude    float64
Altitude     float64
dtype: object


In [ ]:
# All altitude values are being measured in feet
# I will convert the altitude values into meters
feetToMetersConversion = 0.3048
LTDF['Altitude'] = LTDF['Altitude'] * feetToMetersConversion
LTDF

<ipython-input-61-4d29146d0361>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LTDF['Altitude'] = LTDF['Altitude'] * feetToMetersConversion


,UserId,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,walk,1207116566,39.477125,75.989985,1248.4608
1,010,walk,1207116568,39.477083,75.989990,1247.8512
2,010,walk,1207116569,39.477053,75.989980,1247.8512
3,010,walk,1207116570,39.477033,75.989972,1247.8512
4,010,walk,1207116608,39.476977,75.989965,1249.9848
...,...,...,...,...,...,...
5377865,179,subway,1227925767,40.029529,116.411977,88.6968
5377866,179,subway,1227925769,40.029320,116.411975,88.0872
5377867,179,subway,1227925771,40.029111,116.411963,83.8200
5377868,179,subway,1227925773,40.028904,116.411962,83.5152


In [ ]:
# Next I will create a new column called 'Revised Mode'
# where it will be the same as the original 'Mode' column but:
# all 'taxi' values are now under 'car';
# all 'subway' values are now under 'train';
# and all 'run' values are now under 'walk'.

# Map the original 'Mode' values to the revised values
revised_modes_mapping = {
    'taxi': 'car',
    'subway': 'train',
    'run': 'walk'
}

# Replace the original 'Mode' values with the revised values
LTDF['Revised Mode'] = LTDF['Mode'].replace(revised_modes_mapping)

# If a mode is not in the mapping, keep the original mode
LTDF['Revised Mode'].fillna(LTDF['Mode'], inplace=True)

# Rename the 'Mode' column to 'Old Mode' and 'Revised Mode' to 'Mode'
LTDF.rename(columns={'Mode': 'Old Mode', 'Revised Mode': 'Mode'}, inplace=True)

# Reorder the columns
LTDF = LTDF[['UserId', 'Old Mode', 'Mode', 'Timestamp', 'Latitude', 'Longitude', 'Altitude']]

LTDF

,UserId,Old Mode,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,walk,walk,1207116566,39.477125,75.989985,1248.4608
1,010,walk,walk,1207116568,39.477083,75.989990,1247.8512
2,010,walk,walk,1207116569,39.477053,75.989980,1247.8512
3,010,walk,walk,1207116570,39.477033,75.989972,1247.8512
4,010,walk,walk,1207116608,39.476977,75.989965,1249.9848
...,...,...,...,...,...,...,...
5377865,179,subway,train,1227925767,40.029529,116.411977,88.6968
5377866,179,subway,train,1227925769,40.029320,116.411975,88.0872
5377867,179,subway,train,1227925771,40.029111,116.411963,83.8200
5377868,179,subway,train,1227925773,40.028904,116.411962,83.5152


In [ ]:
# There seems to be a single trajectory with a latitude of about 400 degrees (this is not possible)
# Remove rows where latitude > 90
LTDF = LTDF[LTDF['Latitude'] <= 90]

In [ ]:
# We have now done all of the cleaning that we can currently do to this dataframe
# So I will rename it to cltdf (which stands for Cleaned Labelled Trajectories DataFrame)
cltdf = LTDF
cltdf

,UserId,Old Mode,Mode,Timestamp,Latitude,Longitude,Altitude
0,010,walk,walk,1207116566,39.477125,75.989985,1248.4608
1,010,walk,walk,1207116568,39.477083,75.989990,1247.8512
2,010,walk,walk,1207116569,39.477053,75.989980,1247.8512
3,010,walk,walk,1207116570,39.477033,75.989972,1247.8512
4,010,walk,walk,1207116608,39.476977,75.989965,1249.9848
...,...,...,...,...,...,...,...
5377865,179,subway,train,1227925767,40.029529,116.411977,88.6968
5377866,179,subway,train,1227925769,40.029320,116.411975,88.0872
5377867,179,subway,train,1227925771,40.029111,116.411963,83.8200
5377868,179,subway,train,1227925773,40.028904,116.411962,83.5152


In [ ]:
# save cltdf to the cleaned labelled trajectories file path (as a .csv)
cltdf.to_csv(cleanedLabelledTrajectoriesPath, index=False)

### Next Steps: Pre-Processing - Steps 4 to 9: Organise Trajectories into Groups, Calculate Motion Values, and Export a Final Pandas DataFrame